<a href="https://colab.research.google.com/github/Dariusz235607/POI_lab3/blob/main/4_keras_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Lab4 - wielowarstwowa sieć perceptronowa
#!pip install tensorflow

In [ ]:
#Importowanie modułów i pakietów
import numpy as np
from keras import Sequential
from keras.layers import Dense
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
#Wczytanie danych z pliku CSV (z cechami tekstur)
df = pd.read_csv('cechy_tekstur.csv')
X = df.drop(columns=['kategoria']).values  # Wektory cech
y = df['kategoria'].values                 # Etykiety tekstur

print(f"Liczba próbek: {len(X)}")
print(f"Liczba cech: {X.shape[1]}")
print(f"Unikalne kategorie: {np.unique(y)}")

Liczba próbek: 2139
Liczba cech: 72
Unikalne kategorie: ['tekstura1' 'tekstura2' 'tekstura3']


In [ ]:
#Wstępne przetwarzanie

#kodowanie etykiet do liczb
label_encoder = LabelEncoder()
y_int = label_encoder.fit_transform(y)

#zapis jako wektor (one-hot)
onehot_encoder = OneHotEncoder(sparse_output=False)
y_onehot = onehot_encoder.fit_transform(y_int.reshape(-1, 1))

#podział danych na zbiór testowy i treningowy
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.3, random_state=42)
print("\n Przykładowe etykiety wektorów one-hot:")
print(y_onehot[:5])


 Przykładowe etykiety wektorów one-hot:
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [ ]:
#model MLP
model = Sequential([
    #w.ukryta 10 neuronów, aktyw. sigmoid
    Dense(10, activation='sigmoid', input_dim=X.shape[1]),
    #w.wyjściowa 3 neurony(=liczba klas), aktyw. softmax
    Dense(3, activation='softmax')
])

#kompilacja
model.compile(
    loss='categorical_crossentropy',
    optimizer='sgd',  # Stochastic Gradient Descent
    metrics=['accuracy']
)

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 10)             │           730 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 763 (2.98 KB)

 Trainable params: 763 (2.98 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#trening modelu

#trening dla 100 epok
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=10,
    shuffle=True,
    verbose=1  # Pokazuj postęp
)

Epoch 1/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5595 - loss: 0.8073
Epoch 2/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9920 - loss: 0.4617
Epoch 3/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9931 - loss: 0.3299
Epoch 4/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9952 - loss: 0.2501
Epoch 5/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9949 - loss: 0.1978
Epoch 6/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9970 - loss: 0.1592
Epoch 7/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9930 - loss: 0.1443
Epoch 8/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9984 - loss: 0.1127
Epoch 9/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9972 - loss: 0.1041
Epoch 10/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9952 - loss: 0.0973
Epoch 11/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9972 - loss: 0.0837
Epoch 12/100
150/150 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
#Ocena modelu

#Predykcja ze zbioru testoweg
y_pred = model.predict(X_test)

#konwersja powrotna z one-hot na etykiety
y_pred_int = np.argmax(y_pred, axis=1)
y_test_int = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test_int, y_pred_int)
print("Macierz pomyłek:\n", cm)

_, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nDokładność na zbiorze testowym: {accuracy * 100:.2f}%")

21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Macierz pomyłek:
 [[224   0   1]
 [  0 196   0]
 [  0   0 221]]

Dokładność na zbiorze testowym: 99.84%
